## Imports ##

In [32]:
import numpy as np
import pandas as pd
import math
import sys
from numpy import random
from numpy import linalg

## Input ##

In [126]:
n_row = 2000
n_col = 1000
possibilities = [-1, 1]

In [127]:
W = np.random.normal(size = (n_row, n_col)) 
X = np.random.choice(possibilities, n_col)
Y = np.maximum((np.dot(W, X)/math.sqrt(n_col)), 0)
T = 0.1
B = 1/T
n_iter = 20
N = 100

## Helper functions ##

In [4]:
def energy(vector):
    _sum = 0
    for i in range(len(Y)):
        _sum += (Y[i] - np.maximum(0, (np.dot(W, vector)[i])/math.sqrt(n_col)))**2
    return _sum

In [5]:
def error(vector):
    sum = 0
    for i in range(len(vector)):
        sum += (vector[i] - X[i])**2   
    return sum/(4*n_col)

In [ ]:
def Metropolis_chain(dim, n_iter, B, treshold, rate):
    test = np.random.choice(possibilities, dim)
    for _iter in range(n_iter):
        if _iter > n_iter*treshold:
            B = B*rate
        to_flip = np.random.randint(0, dim)
        to_test = test.copy()
        to_test[to_flip] = -to_test[to_flip]
        proba = np.minimum(1, np.exp(-B*(energy(to_test)-energy(test))))
        print("proba : ", proba)
        if random.random() < proba:
            test = to_test
    return error(test)    

In [133]:
def Glauber(dim, n_iter, B, treshold, rate):
    test = np.random.choice(possibilities, dim)
    for _iter in range(n_iter):
        if _iter % 10 == 0 :
            print(error(test))
        if _iter > n_iter*treshold:
            B = B*rate
        to_flip = np.random.randint(0, dim)
        flipped = test.copy()
        flipped[to_flip] = test[to_flip]*-1
        proba = (1 + test[to_flip]*math.tanh(B*(energy(flipped) - energy(test))))/2.0
        print("proba : ", proba)
        if random.random() < proba:
            test[to_flip] = 1 
        else:
            test[to_flip] = -1
    return error(test)

## Optimization ##

In [8]:
t_tresholds = np.linspace(0.50, 0.95, num=10)
t_increasing = np.linspace(1.5, 5, num=8)
best_error = 1e6
best_set_up = None
for treshold in t_tresholds:
    for increasing in t_increasing:
        for test in range(10):
            metro_results = []
            glauber_results = []
            metro_error = Metropolis_chain(n_col, n_iter, B, treshold, increasing)
            glauber_error = Glauber(n_col, n_iter, B, treshold, increasing)
            metro_results.append(metro_error)
            glauber_results.append(glauber_error)
        metro_mean = np.mean(metro_results)
        if(metro_mean < best_error):
            best_error = metro_mean
            best_set_up = ("Metropolis", "treshold :",treshold, "rate :", increasing, "error :", best_error)
        glauber_mean = np.mean(glauber_results)
        if(glauber_mean < best_error):
            best_error = glauber_mean
            best_set_up = ("Glauber " ,"treshold :",treshold, "rate :", increasing, "error :", best_error)

KeyboardInterrupt: 

In [1]:
best_set_up

NameError: name 'best_set_up' is not defined

## Results ##  

In [117]:
Metropolis_chain(n_col, 100000, B, 0.6, 1.5)

proba :  1.0
proba :  1.0
proba :  1.0
proba :  1.0
proba :  1.0
proba :  1.0
proba :  1.0
proba :  1.0
proba :  1.0
proba :  1.0
proba :  1.0
proba :  1.0
proba :  1.0
proba :  1.0
proba :  1.0
proba :  1.0
proba :  1.0
proba :  1.0
proba :  1.0
proba :  1.0
proba :  1.0
proba :  1.0
proba :  1.0
proba :  1.0
proba :  1.0
proba :  1.0
proba :  1.0
proba :  1.0
proba :  1.0
proba :  1.0
proba :  1.0
proba :  1.0
proba :  1.0
proba :  1.0
proba :  1.0
proba :  1.0
proba :  1.0
proba :  1.0
proba :  1.0
proba :  1.0
proba :  1.0
proba :  1.0
proba :  1.0
proba :  1.0
proba :  1.0
proba :  1.0
proba :  1.0
proba :  1.0
proba :  1.0
proba :  1.0
proba :  1.0
proba :  1.0
proba :  1.0
proba :  1.0
proba :  1.0
proba :  1.0
proba :  1.0
proba :  1.0
proba :  1.0
proba :  1.0
proba :  1.0
proba :  1.0
proba :  1.0
proba :  1.0
proba :  1.0
proba :  1.0
proba :  1.0
proba :  1.0
proba :  1.0
proba :  1.0
proba :  1.0
proba :  1.0
proba :  1.0
proba :  1.0
proba :  1.0
proba :  1.0
proba :  1.0

proba :  1.0
proba :  1.0
proba :  1.0
proba :  1.0
proba :  1.0
proba :  1.0
proba :  1.0
proba :  1.0
proba :  1.0
proba :  1.0
proba :  1.0
proba :  1.0
proba :  1.0
proba :  1.0
proba :  1.0
proba :  1.0
proba :  1.0
proba :  1.0
proba :  1.0
proba :  1.0
proba :  1.0
proba :  1.0
proba :  1.0
proba :  1.0
proba :  1.0
proba :  1.0
proba :  1.0
proba :  1.0
proba :  1.0
proba :  1.0
proba :  1.0
proba :  1.0
proba :  1.0
proba :  1.0
proba :  1.0
proba :  1.0
proba :  1.0
proba :  1.0
proba :  1.0
proba :  1.0
proba :  1.0
proba :  1.0
proba :  1.0
proba :  1.0
proba :  1.0
proba :  1.0
proba :  1.0
proba :  1.0
proba :  1.0
proba :  1.0
proba :  1.0
proba :  1.0
proba :  1.0
proba :  1.0
proba :  1.0
proba :  1.0
proba :  1.0
proba :  1.0
proba :  1.0
proba :  1.0
proba :  1.0
proba :  1.0
proba :  1.0
proba :  1.0
proba :  1.0
proba :  1.0
proba :  1.0
proba :  1.0
proba :  1.0
proba :  1.0
proba :  1.0
proba :  1.0
proba :  1.0
proba :  1.0
proba :  1.0
proba :  1.0
proba :  1.0

proba :  1.0
proba :  1.0
proba :  1.0
proba :  1.0
proba :  1.0
proba :  1.0
proba :  1.0
proba :  1.0
proba :  1.0
proba :  1.0
proba :  1.0
proba :  1.0
proba :  1.0
proba :  1.0
proba :  1.0
proba :  1.0
proba :  1.0
proba :  1.0
proba :  1.0
proba :  1.0
proba :  1.0
proba :  1.0
proba :  1.0
proba :  1.0
proba :  1.0
proba :  1.0
proba :  1.0
proba :  1.0
proba :  1.0
proba :  1.0
proba :  1.0
proba :  1.0
proba :  1.0
proba :  1.0
proba :  1.0
proba :  1.0
proba :  1.0
proba :  1.0
proba :  1.0
proba :  1.0
proba :  1.0
proba :  1.0
proba :  1.0
proba :  1.0
proba :  1.0
proba :  1.0
proba :  1.0
proba :  1.0
proba :  1.0
proba :  1.0
proba :  1.0
proba :  1.0
proba :  1.0
proba :  1.0
proba :  1.0
proba :  1.0
proba :  1.0
proba :  1.0
proba :  1.0
proba :  1.0
proba :  1.0
proba :  1.0
proba :  1.0
proba :  1.0
proba :  1.0
proba :  1.0
proba :  1.0
proba :  1.0
proba :  1.0
proba :  1.0
proba :  1.0
proba :  1.0
proba :  1.0
proba :  1.0
proba :  1.0
proba :  1.0
proba :  1.0

proba :  1.0
proba :  1.0
proba :  1.0
proba :  1.0
proba :  1.0
proba :  1.0
proba :  1.0
proba :  1.0
proba :  1.0
proba :  1.0
proba :  1.0
proba :  1.0
proba :  1.0
proba :  1.0
proba :  1.0
proba :  1.0
proba :  1.0
proba :  1.0
proba :  1.0
proba :  1.0
proba :  1.0
proba :  1.0
proba :  1.0
proba :  1.0
proba :  1.0
proba :  1.0
proba :  1.0
proba :  1.0
proba :  1.0
proba :  1.0
proba :  1.0
proba :  1.0
proba :  1.0
proba :  1.0
proba :  1.0
proba :  1.0
proba :  1.0
proba :  1.0
proba :  1.0
proba :  1.0
proba :  1.0
proba :  1.0
proba :  1.0
proba :  1.0
proba :  1.0
proba :  1.0
proba :  1.0
proba :  1.0
proba :  1.0
proba :  1.0
proba :  1.0
proba :  1.0
proba :  1.0
proba :  1.0
proba :  1.0
proba :  1.0
proba :  1.0
proba :  1.0
proba :  1.0
proba :  1.0
proba :  1.0
proba :  1.0
proba :  1.0
proba :  1.0
proba :  1.0
proba :  1.0
proba :  1.0
proba :  1.0
proba :  1.0
proba :  1.0
proba :  1.0
proba :  1.0
proba :  1.0
proba :  1.0
proba :  1.0
proba :  1.0
proba :  1.0

proba :  1.0
proba :  1.0
proba :  1.0
proba :  1.0
proba :  1.0
proba :  1.0
proba :  1.0
proba :  1.0
proba :  1.0
proba :  1.0
proba :  1.0
proba :  1.0
proba :  1.0
proba :  1.0
proba :  1.0
proba :  1.0
proba :  1.0
proba :  1.0
proba :  1.0
proba :  1.0
proba :  1.0
proba :  1.0
proba :  1.0
proba :  1.0
proba :  1.0
proba :  1.0
proba :  1.0
proba :  1.0
proba :  1.0
proba :  1.0
proba :  1.0
proba :  1.0
proba :  1.0
proba :  1.0
proba :  1.0
proba :  1.0
proba :  1.0
proba :  1.0
proba :  1.0
proba :  1.0
proba :  1.0
proba :  1.0
proba :  1.0
proba :  1.0
proba :  1.0
proba :  1.0
proba :  1.0
proba :  1.0
proba :  1.0
proba :  1.0
proba :  1.0
proba :  1.0
proba :  1.0
proba :  1.0
proba :  1.0
proba :  1.0
proba :  1.0
proba :  1.0
proba :  1.0
proba :  1.0
proba :  1.0
proba :  1.0
proba :  1.0
proba :  1.0
proba :  1.0
proba :  1.0
proba :  1.0
proba :  1.0
proba :  1.0
proba :  1.0
proba :  1.0
proba :  1.0
proba :  1.0
proba :  1.0
proba :  1.0
proba :  1.0
proba :  1.0

...

proba :  nan
proba :  nan
proba :  nan
proba :  nan
proba :  nan
proba :  nan
proba :  nan
proba :  nan
proba :  nan
proba :  nan
proba :  nan
proba :  nan
proba :  nan
proba :  nan
proba :  nan
proba :  nan
proba :  nan
proba :  nan
proba :  nan
proba :  nan
proba :  nan
proba :  nan
proba :  nan
proba :  nan
proba :  nan
proba :  nan
proba :  nan
proba :  nan
proba :  nan
proba :  nan
proba :  nan
proba :  nan
proba :  nan
proba :  nan
proba :  nan
proba :  nan
proba :  nan
proba :  nan
proba :  nan
proba :  nan
proba :  nan
proba :  nan
proba :  nan
proba :  nan
proba :  nan
proba :  nan
proba :  nan
proba :  nan
proba :  nan
proba :  nan
proba :  nan
proba :  nan
proba :  nan
proba :  nan
proba :  nan
proba :  nan
proba :  nan
proba :  nan
proba :  nan
proba :  nan
proba :  nan
proba :  nan
proba :  nan
proba :  nan
proba :  nan
proba :  nan
proba :  nan
proba :  nan
proba :  nan
proba :  nan
proba :  nan
proba :  nan
proba :  nan
proba :  nan
proba :  nan
proba :  nan
proba :  nan

proba :  nan
proba :  nan
proba :  nan
proba :  nan
proba :  nan
proba :  nan
proba :  nan
proba :  nan
proba :  nan
proba :  nan
proba :  nan
proba :  nan
proba :  nan
proba :  nan
proba :  nan
proba :  nan
proba :  nan
proba :  nan
proba :  nan
proba :  nan
proba :  nan
proba :  nan
proba :  nan
proba :  nan
proba :  nan
proba :  nan
proba :  nan
proba :  nan
proba :  nan
proba :  nan
proba :  nan
proba :  nan
proba :  nan
proba :  nan
proba :  nan
proba :  nan
proba :  nan
proba :  nan
proba :  nan
proba :  nan
proba :  nan
proba :  nan
proba :  nan
proba :  nan
proba :  nan
proba :  nan
proba :  nan
proba :  nan
proba :  nan
proba :  nan
proba :  nan
proba :  nan
proba :  nan
proba :  nan
proba :  nan
proba :  nan
proba :  nan
proba :  nan
proba :  nan
proba :  nan
proba :  nan
proba :  nan
proba :  nan
proba :  nan
proba :  nan
proba :  nan
proba :  nan
proba :  nan
proba :  nan
proba :  nan
proba :  nan
proba :  nan
proba :  nan
proba :  nan
proba :  nan
proba :  nan
proba :  nan

proba :  nan
proba :  nan
proba :  nan
proba :  nan
proba :  nan
proba :  nan
proba :  nan
proba :  nan
proba :  nan
proba :  nan
proba :  nan
proba :  nan
proba :  nan
proba :  nan
proba :  nan
proba :  nan
proba :  nan
proba :  nan
proba :  nan
proba :  nan
proba :  nan
proba :  nan
proba :  nan
proba :  nan
proba :  nan
proba :  nan
proba :  nan
proba :  nan
proba :  nan
proba :  nan
proba :  nan
proba :  nan
proba :  nan
proba :  nan
proba :  nan
proba :  nan
proba :  nan
proba :  nan
proba :  nan
proba :  nan
proba :  nan
proba :  nan
proba :  nan
proba :  nan
proba :  nan
proba :  nan
proba :  nan
proba :  nan
proba :  nan
proba :  nan
proba :  nan
proba :  nan
proba :  nan
proba :  nan
proba :  nan
proba :  nan
proba :  nan
proba :  nan
proba :  nan
proba :  nan
proba :  nan
proba :  nan
proba :  nan
proba :  nan
proba :  nan
proba :  nan
proba :  nan
proba :  nan
proba :  nan
proba :  nan
proba :  nan
proba :  nan
proba :  nan
proba :  nan
proba :  nan
proba :  nan
proba :  nan

proba :  nan
proba :  nan
proba :  nan
proba :  nan
proba :  nan
proba :  nan
proba :  nan
proba :  nan
proba :  nan
proba :  nan
proba :  nan
proba :  nan
proba :  nan
proba :  nan
proba :  nan
proba :  nan
proba :  nan
proba :  nan
proba :  nan
proba :  nan
proba :  nan
proba :  nan
proba :  nan
proba :  nan
proba :  nan
proba :  nan
proba :  nan
proba :  nan
proba :  nan
proba :  nan
proba :  nan
proba :  nan
proba :  nan
proba :  nan
proba :  nan
proba :  nan
proba :  nan
proba :  nan
proba :  nan
proba :  nan
proba :  nan
proba :  nan
proba :  nan
proba :  nan
proba :  nan
proba :  nan
proba :  nan
proba :  nan
proba :  nan
proba :  nan
proba :  nan
proba :  nan
proba :  nan
proba :  nan
proba :  nan
proba :  nan
proba :  nan
proba :  nan
proba :  nan
proba :  nan
proba :  nan
proba :  nan
proba :  nan
proba :  nan
proba :  nan
proba :  nan
proba :  nan
proba :  nan
proba :  nan
proba :  nan
proba :  nan
proba :  nan
proba :  nan
proba :  nan
proba :  nan
proba :  nan
proba :  nan

KeyboardInterrupt: 

In [140]:
Glauber(n_col, 50, 2, 0.6, 1.5)

0.532
proba :  0.9999999969677252
proba :  0.9999999999689677
proba :  0.9048655849825455
proba :  0.9999996534582658
proba :  4.136369025076192e-11
proba :  4.002460372576344e-07
proba :  2.258045696496014e-05
proba :  0.9992597791332072
proba :  0.9999999999989365
proba :  9.033785819773099e-05
0.531
proba :  0.9999999663904524
proba :  0.006215608366928804
proba :  0.9653065624587549
proba :  2.925672942799551e-08
proba :  0.9999873196312277
proba :  0.9741691591823247
proba :  0.9999999999995641
proba :  0.0067478816987177614
proba :  0.0010309232509262412
proba :  0.08773593388455869
0.53
proba :  2.905564677746497e-12
proba :  0.9999999999996771
proba :  0.995566670511306
proba :  1.792443471426708e-09
proba :  0.9999997708213348
proba :  0.9981689732489778
proba :  0.5514871309152889
proba :  0.03576589778122652
proba :  0.999999999944321
proba :  0.9624819902728038
0.531
proba :  0.9999999901294736
proba :  0.033522780491612
proba :  1.0177119008636382e-07
proba :  0.9999999999

0.525

In [138]:
values = []
B_values = np.linspace(1, 10, num = 5)
for b in B_values:
    for i in range(3):
        values.append((b,Glauber(n_col, 50, b, 0.75, 1.75)))

0.496
proba :  0.9845489830287478
proba :  0.8422831403135678
proba :  0.9999889196373277
proba :  0.994215504899786
proba :  0.00038431592612742094
proba :  0.9883352936708885
proba :  0.8753694779332746
proba :  0.9735996411961392
proba :  0.0026808444732746017
proba :  0.46471755638484885
0.494
proba :  0.964639014329487
proba :  0.00033072058389144177
proba :  0.9821934076781726
proba :  0.9955446172283464
proba :  0.9999999995792042
proba :  0.001799992561690944
proba :  0.9999999342712989
proba :  0.9999999028801558
proba :  3.13499034398923e-07


KeyboardInterrupt: 

In [11]:
t_tresholds

array([0.5 , 0.55, 0.6 , 0.65, 0.7 , 0.75, 0.8 , 0.85, 0.9 , 0.95])

In [ ]:
errors = []
for i in range(N):
    error = #Glauber ou Metropolis_chain selon best_set_up
    errors.append(error)
mean = np.mean(errors)
variance = np.var(errors)
print('Mean error : ', mean)
print('Variance of error : ' variance)

In [86]:
best_set_up

('Metropolis', 'treshold :', 0.5, 'rate :', 5.0, 'error :', 0.015)

In [82]:
t_increasing = np.linspace(1.5, 5, num=8)
t_increasing

array([1.5, 2. , 2.5, 3. , 3.5, 4. , 4.5, 5. ])

In [ ]:
BEST_SO_FAR = 0.01411756195508118 #Metropolis_chain(n_col, 100, B) 0.8